In [1]:
import pandas as pd

from faltwerk.io import get_alphafold2_model_from_ena
from faltwerk.models import Fold, Binding
from faltwerk.stats import find_hotspots
from faltwerk.vis import Layout


In [2]:
pfam = '.../pfam_v31/Pfam-A.hmm'
ID = 'AF-P35573-F1'

In [3]:
_ = get_alphafold2_model_from_ena(ID, 'example.pdb')
model = Fold('example.pdb')


In [4]:
clinvar = pd.read_csv('clinvar.tsv', sep='\t')
select = [int(i) for i in clinvar[clinvar['ID'] == ID]['positions'].item().split(',')]


In [5]:
mut = [1 if i in select else 0 for i in range(len(model))]
model.annotate_('mut', mut)
model.annotate_('hotspot', find_hotspots(model, mut))


In [6]:
b = Binding(model, 'representable')
b.predict_binding_(pfam)
# b.domains
# b.interactions

In [7]:
b.ligands['PF14702.5']

{'ALL_', 'DRUGLIKE_', 'METABOLITE_', 'MTT', 'SM_'}

In [8]:
binding = b.get_binding('PF14702.5', 'ALL_')
ligands = [i for i, j in enumerate(binding) if j > .5]


In [12]:
ly = Layout(model, grid=(1, 2), panel_size=(250, 250))

x1 = ly.select(residues=select, elements=['CA'])
x2 = ly.select(residues=[567],  elements=['CA'])
lg = ly.select(residues=ligands)

ly.geom_ribbon(opacity=.5)

ly.geom_sphere(selection=x1, color='black')
ly.geom_sphere(selection=x2, color='red', radius=3)
ly.geom_ribbon(selection=lg, color='red')

ly.geom_surface('hotspot', palette='binary', panel=(0, 1), opacity=.75)
ly.geom_surface(selection=lg, color='red', panel=(0, 1))

ly.render().show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [68]:
ly.geom_sphere?
